In [2]:
import cv2
import os
from sklearn.metrics import accuracy_score

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Function to detect keypoints and descriptors using SIFT
def detect_keypoints(image_path):
    image = cv2.imread(image_path, 0)  # Read the image as grayscale
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

# Load images from training and testing directories
train_path = r'M:\SIFT\sm-train'
test_path = r'M:\SIFT\sm-train'

train_images = []
train_descriptors = []
labels = []

# Train on the training set
for class_folder in os.listdir(train_path):
    class_dir = os.path.join(train_path, class_folder)
    if os.path.isdir(class_dir):
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            _, desc = detect_keypoints(image_path)
            if desc is not None:
                train_images.append(image_path)
                train_descriptors.append(desc)
                labels.append(class_folder)

# Match test images against training set
bf = cv2.BFMatcher()
y_true = []
y_pred = []

for class_folder in os.listdir(test_path):
    class_dir = os.path.join(test_path, class_folder)
    if os.path.isdir(class_dir):
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            kp_test, desc_test = detect_keypoints(image_path)
            if desc_test is not None:
                best_class = None
                best_match_count = 0
                for i, train_desc in enumerate(train_descriptors):
                    # Perform the matching with KNN
                    matches = bf.knnMatch(desc_test, train_desc, k=2)
                    
                    # Apply ratio test to find good matches
                    good_matches = []
                    for match in matches:
                        if len(match) == 2:  # Ensure there are at least two neighbors
                            m, n = match
                            if m.distance < 0.75 * n.distance:
                                good_matches.append(m)
                    
                    # Check the number of good matches
                    if len(good_matches) > best_match_count:
                        best_match_count = len(good_matches)
                        best_class = labels[i]

                # Append results
                y_true.append(class_folder)
                y_pred.append(best_class if best_class else "Unknown")

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Detection Accuracy: {accuracy * 100:.2f}%")


Detection Accuracy: nan%


c:\Users\CSE-AiD\anaconda3\envs\py310\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\CSE-AiD\anaconda3\envs\py310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
